In [1]:
import pandas as pd
from math import ceil, floor
import matplotlib.pyplot as plt
import seaborn as sns
import json
import matplotlib
from matplotlib import ticker
%matplotlib inline

In [2]:
data = pd.read_csv("../components/artifacts/postprocess_data.csv", sep = "\t", index_col = 0)
data["Date"] = pd.to_datetime(data["Date"])
data["Gender"] = data["Gender"].astype("category")
data["Account Type"] = data["Account Type"].astype("category")
data["Tweet Type"] = pd.Categorical(values = data["Tweet Type"], categories = ["share", "reply", "post"], ordered = True)

/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
data.head()

,Date,Page Type,Author,Full Name,Full Text,Hashtags,Impact,Impressions,Tweet Type,Thread Author,...,topic_9,topic_10,screen,Gender,Account Type,followers_count,screen_originator,Gender_originator,Account Type_originator,followers_count_originator
0,2017-12-06 23:59:59,twitter,William82861606,William82861606 (William),funder conyers resigned franken resigning tomo...,"#trumpsexprobe, #metoo",0.0,282.0,share,funder,...,0.0,0.000000,William82861606,male,core,2548.0,funder,male,influencer,336269.0
1,2017-12-06 23:59:58,twitter,Artis_Elemental,Artis_Elemental (ART...Is),alyssa_milano list sexual misconduct allegatio...,#metoo,24.5,5578.0,share,Alyssa_Milano,...,0.0,0.000000,Artis_Elemental,male,core,7217.0,Alyssa_Milano,female,influencer,3698114.0
2,2017-12-06 23:59:57,twitter,lanah03,lanah03 (Laura Hopkins),deepinthehills sobbing silent longer,#metoo,5.4,2713.0,share,DeepInTheHills,...,0.0,0.000000,lanah03,female,core,2775.0,NaN,NaN,NaN,NaN
3,2017-12-06 23:59:55,twitter,lidskawasaki,lidskawasaki (dearlordbabyjesus),alyssa_milano time asking franken resign senat...,"#timepersonoftheyear, #metoo",0.0,330.0,share,Alyssa_Milano,...,0.0,0.444376,lidskawasaki,female,core,655.0,Alyssa_Milano,female,influencer,3698114.0
4,2017-12-06 23:59:54,twitter,IAmMeliLopez,IAmMeliLopez (Melissa Lopez),nycchr public hearing begun stories deserve he...,"#sexualharassment, #metoo",0.0,13.0,share,NYCCHR,...,0.0,0.000000,IAmMeliLopez,female,core,18.0,NYCCHR,male,core,12572.0


In [6]:
data.columns


Index(['Date', 'Page Type', 'Author', 'Full Name', 'Full Text', 'Hashtags',
       'Impact', 'Impressions', 'Tweet Type', 'Thread Author',
       'Twitter Followers', 'Twitter Following', 'Twitter Tweets',
       'Twitter Reply Count', 'Twitter Verified', 'Twitter Retweets',
       'Reach (new)', 'Region', 'topic_1', 'topic_2', 'topic_3', 'topic_4',
       'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10',
       'screen', 'Gender', 'Account Type', 'followers_count',
       'screen_originator', 'Gender_originator', 'Account Type_originator',
       'followers_count_originator'],
      dtype='object')

In [10]:
topic_score_cols = [x for x in data.columns if 'topic' in x]
topic_data = data[['Date','Author','Page Type','Account Type','Full Text'] + topic_score_cols]
topic_data['Topic'] = topic_data[topic_score_cols].idxmax(axis=1)

topic_data.head()

<ipython-input-10-893862cee2d3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topic_data['Topic'] = topic_data[topic_score_cols].idxmax(axis=1)


,Date,Author,Page Type,Account Type,Full Text,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,Topic
0,2017-12-06 23:59:59,William82861606,twitter,core,funder conyers resigned franken resigning tomo...,0.000000,0.996263,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,topic_2
1,2017-12-06 23:59:58,Artis_Elemental,twitter,core,alyssa_milano list sexual misconduct allegatio...,0.000000,0.990108,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,topic_2
2,2017-12-06 23:59:57,lanah03,twitter,core,deepinthehills sobbing silent longer,0.702286,0.000000,0.0,0.0,0.000000,0.000000,0.277947,0.0,0.0,0.000000,topic_1
3,2017-12-06 23:59:55,lidskawasaki,twitter,core,alyssa_milano time asking franken resign senat...,0.000000,0.000000,0.0,0.0,0.000000,0.551623,0.000000,0.0,0.0,0.444376,topic_6
4,2017-12-06 23:59:54,IAmMeliLopez,twitter,core,nycchr public hearing begun stories deserve he...,0.000000,0.000000,0.0,0.0,0.991076,0.000000,0.000000,0.0,0.0,0.000000,topic_5


In [20]:
for topic in topic_score_cols:
    print(f'TOPIC {topic}')
    display(topic_data[topic_data['Topic'] == topic][['Date','Author','Page Type','Account Type','Full Text']].sample(20))
    display('-'*30)

TOPIC topic_1


,Date,Author,Page Type,Account Type,Full Text
337616,2018-10-03 22:49:02,GreenIHouse,twitter,organisational,thedemcoalition weve letters sent key undecide...
1342865,2018-09-21 16:00:37,LisaADavis1,twitter,core,paulrgordon stories whyididntreport horrible e...
6575126,2017-11-01 19:27:52,BassGuitarHero,twitter,core,rosemcgowan stand bridie ve hurt deeply scarle...
1622756,2018-09-27 18:16:07,aunt_bitchyb,twitter,core,thekoukoureport dr ford thankfully dead buried...
6753572,2019-07-07 03:37:27,carrie_dixonLHC,twitter,core,sfpelosi epstein case horrific young women des...
861552,2018-02-05 16:16:53,MissyJo79,twitter,influencer,amymek media liberal blackout real movement he...
4395863,2018-09-07 22:40:51,pirateonboard,twitter,core,dairyblondeln realdonaldtrump un indicted co c...
2571175,2018-04-27 16:08:06,tubmansghost,twitter,core,era cosby teams attacks victims backfired
3052461,2018-02-11 21:02:15,ebwobusy,twitter,core,connerhabib practically begging hear movement ...
3688828,2018-06-09 19:45:03,Modeefr67,twitter,core,anon_decoder brave brave woman inside belly be...


'------------------------------'

TOPIC topic_2


,Date,Author,Page Type,Account Type,Full Text
6129802,2019-09-08 12:11:03,d_nurnberg,twitter,core,pinkkfrogg hollywood people sick alyssa_milano
3155707,2018-02-13 00:28:25,p143aris,twitter,core,jacob kids father amazing daughters son respon...
5155905,2018-03-01 14:41:06,LoriHandrahan2,twitter,organisational,jimgamble_ineqe bad aid sexual abuse aid indus...
6444969,2017-11-11 19:45:23,danielle2u,twitter,core,zorrah wolfe thank glad assaults blessings pea...
5807360,2020-03-05 15:59:28,AmysandersonAmy,twitter,core,wolverinedg kathygriffin didn remember entire ...
4221458,2018-10-19 14:27:48,JamiJm,twitter,core,mog maddow busts trump fbi boost dc hotel prof...
5901144,2018-03-26 00:54:44,rrey2101,twitter,core,joshdcaplan obamas crack head lover larry sinc...
5663285,2017-12-04 00:06:53,Bernie4Revo,twitter,core,buzzfeednews black woman incredible statement ...
7023372,2017-11-20 14:47:35,YogiSunlight,twitter,core,yogisunlight meetthepress hughhewitt hopefully...
3344039,2018-01-20 15:03:39,ACCIDENTALDIVA,twitter,influencer,like status followed chased public stranger said


...

,Date,Author,Page Type,Account Type,Full Text
3902601,2018-05-24 19:22:01,SojournerNow,twitter,core,restlessnews cynthianixon co_rapunzel citizens...
2582920,2018-04-26 23:49:29,StaceyDesro,twitter,core,ananavarro hard parts supporting movement cons...
6581738,2017-10-31 13:05:27,wolfgangfaustX,twitter,core,rosemcgowan janis paige kept pain secret years...
1376223,2018-09-22 20:08:34,mikandynothem,twitter,influencer,mikandynothem sengillibrand hugging worst offe...
999783,2020-05-02 00:06:18,DaveChristenso8,twitter,core,mercedesschlapp democratic women senators ask ...
5443276,2019-03-31 17:18:19,gailslpmosaic,twitter,core,usatoday joebiden lucyflores lucy flores makin...
2525866,2020-04-26 10:53:18,chuxter7,twitter,core,realcandaceo woman attacked brett kavanaugh ze...
6619233,2017-10-26 00:48:36,xolanacho,twitter,core,short story girlhood women color disillusionme...
3220702,2018-02-19 03:03:48,ChelarVale,twitter,core,national anthem fergie bad timesup forefront m...
7004282,2017-11-21 23:14:42,ajain31,twitter,core,dropthe_mic heroine thats stop sexualassault s...


'------------------------------'

TOPIC topic_10


,Date,Author,Page Type,Account Type,Full Text
1537639,2018-09-25 14:30:06,JeremiahHorriga,twitter,core,marzclaire thisderekhauk aodespair moon came c...
1172890,2018-09-17 01:32:07,PaulaVotes,twitter,core,dont vote
6217446,2017-12-08 17:17:15,TheStandWA,twitter,organisational,friday stand sounds silence rich heir vs kids ...
2698509,2017-12-15 21:55:27,mamarose50hotma,twitter,core,realsaavedra twitter facebook keeping report l...
3327162,2018-01-20 23:31:35,powzorro,twitter,core,dagmarwoyde yes differences degrees billmaher ...
6566380,2018-12-01 13:59:56,LarryCrnobrnja,twitter,core,star cle heads butts banning itscoldoutside mi...
4136062,2018-10-15 17:15:35,DarylForsberg,twitter,core,gerfingerpoken shut mazie hirono said believe ...
3880784,2018-05-26 05:51:46,TargetedinMO,twitter,organisational,lauraloomer harveyweinstein surrendered nypd m...
1233928,2018-09-19 21:38:47,Strikertrader1,twitter,core,farce feminism hypocrisy metoowitchhunt select...
5087146,2019-02-23 12:08:51,timdix,twitter,core,thedailyedge new trump like predator teens par...


'------------------------------'

In [22]:
for topic in topic_score_cols:
    print(f'TOPIC {topic}')
    display('|'.join(topic_data[topic_data['Topic'] == topic]['Full Text'].sample(30).tolist()))
    display('-'*30)

TOPIC topic_1


'concern isnt kavanaughs accusers credible concern gop movement wont care theyre approving sexual predator nations highest court|cnni tamara burke created movement years ago moment men women people survivors holding stories pit stomach heard believed cnntownhall|sthtx_ platanoman people trying yell hes holding crazy person like person needs jail|amy_siskind waking story cheating trump baron born payoffs silence man disgrace country realdonaldtrump resign|juliabhaber entered flsd docket timesup pdf|ppmaryland february national teen dating violence awareness month month long ll join loveisrespect raising awareness dating violence prevention loveisrespect ntdvam|essence started work resources recourses november taranaburke founder guest editor special issue essence celebrating fight end sexual violence stands|op ed columnist sick stomach|lindsay lohan thinks strong women look weak|chsommers read superb expose twitter feminism excesses eloquent katie roiphe search twitter behold twitter fe

'------------------------------'

TOPIC topic_2


'lightfootinhwd truth coming cleopatra farrisflagg catslawrence lesmoonves exposed revealed|rosemcgowan funder igotroymoored new|alyssa_milano fifth americans know said|combination women lot control having voice freedom campaign kind merged opened sex industry emma sayle|movement targets german language sexist|l_u_cy fhmertz rape survivor support men ask ohio state wrestlers ask terry crews plenty men assaulted life need support|dashannestokes americans gop national security experts psychiatrists conference yale trump unfit office resist theresistance trumprussia saturdaymorning maga trump|kerryreid au pair mad men deserves fictional moment|teriwilsonauthr great morning sara_rwa meeting nicoleflockton came houston talk abt writing romance wake movement presentation included passages collection classic millsandboon romances ty sharing weekend nikki|govmikehuckabee yeah|mishacollins thing ask blindly accept woman assaulted claim ask claims seriously time|brittiself everyday carry gun kni

...

'spitfiretyler alyssa_milano sengillibrand speakerpelosi aoc rashidatlaib maziehirono hell believing women clear applies fits political agenda tarareade credible corroborating evidence|sbarolo feminism ruining|ve seen james franco news lately join episode brave victim violetpaley comes truth try stay objective hard digest listen open ears open heart timesup|gamora trump wanted wall wall women dr ford stand protecting stand solidarity stand support|realsaavedra twitter facebook keeping report lisa bloom offering pay women considered making sexual misconduct allegations donald trump final months presidential race trending sections lisabloom|realcandaceo pic hillaryclinton dining harvey weinstein weeks election loss fun fact hollywood women supported withher surely expressing outrage shortly|guardian ellen barkin tweets accusation terry gilliam directors comments|andrealearned whoa started teaching end august sexual harassment began september michiganstateu jenniferdillpsu important read 

'------------------------------'

TOPIC topic_10


'weve received money victims eviction record says beth frank housing discrimination|anncapt yashar samhaskell lynnweidner miss america pageant kinda lessens movement people pageant men parading stage asked questions wear swimsuits|feministchina check new post metooinchina activist pinerpiner xiao meili discusses develops butterflies hurricane|sallyalbright publicly privately misgivings|nydems woman said countless stories told gov cuomo message ny listening|fastcompany happen|thestreet talks dr gilda warning fraternization means career suicide days askdrgilda drgilda phd professor careers careeradvice careertips ceo intel briankrzanich boardmember|today wearing black support timesup whyididntreport firstselfie brielarson|sjgrunewald twitter amazing himtoo|theauthor_bjm k_jeanpierre collectively win support ty support retweet new book god gift inspirational motivational women quotes advice timesup|midnightsdarlin wtf yall forgotten corey_feldman jamieleecurtis elizadushku corey tweeting 

'------------------------------'